# Loading Images and Masks from DICOMs

In [2]:
from pathlib import Path

from imgtools.logging import logger
from imgtools.ops.input_classes import ImageMaskInput, ImageMaskModalities

logger.setLevel("ERROR")

## Getting Started

You need the following at minimum:
- Path to the directory containing the DICOM files
- Establish which Image and Mask modalities you want to use:
Combinations are:  

| Image Modality | Mask Modality |
|----------------|---------------|
| CT             | RTSTRUCT      |
| MR             | RTSTRUCT      |
| CT             | SEG      |
| MR             | SEG      |

For this tutorial we will download two datasets:


In [3]:
# Define the path to the data
testdata = Path("testdata")
# for this tutorial we will use some test image data
datasets_name = ["NSCLC-Radiomics", "Vestibular-Schwannoma-SEG"]

In [ ]:
%%capture 
# download data using the imgtools cli
for ds in datasets_name:
    if not (testdata / f"{ds}.tar.gz").exists():
        print(f"Downloading {ds} dataset")
        !imgtools testdata -a {ds}.tar.gz {testdata.absolute()}


## Setting up the loaders with `ImageMaskInput`

### Vestibular-Schwannoma-SEG

has MR as scan and RTSTRUCT as mask:


| Patient ID           | Modality  | Number of Series    |
|----------------------|-----------|---------------------|
| VS-SEG-001           | MR        | 2                   |
| VS-SEG-001           | RTSTRUCT  | 2                   |
| VS-SEG-002           | MR        | 2                   |
| VS-SEG-002           | RTSTRUCT  | 2                   |

In [5]:
vs_seg = ImageMaskInput(
  dir_path=testdata / datasets_name[1],
  modalities=ImageMaskModalities.MR_RTSTRUCT
)

print(vs_seg)

ImageMaskInput<
	num_cases=4,
	dataset_name='Vestibular-Schwannoma-SEG',
	modalities=<MR,RTSTRUCT>,
	output_streams=['MR', 'RTSTRUCT_MR'],
	series_col_names=['series_MR', 'series_RTSTRUCT_MR'],
>


### NSCLC-Radiomics

has CT as scan and BOTH RTSTRUCT and SEG as masks.

| Patient ID           | Modality  | Number of Series    |
|----------------------|-----------|---------------------|
| LUNG1-001            | CT        | 1                   |
| LUNG1-001            | RTSTRUCT  | 1                   |
| LUNG1-001            | SEG       | 1                   |
| LUNG1-002            | CT        | 1                   |
| LUNG1-002            | RTSTRUCT  | 1                   |
| LUNG1-002            | SEG       | 1                   |

In [10]:
nsclsc_rtstruct = ImageMaskInput(
    dir_path=testdata / datasets_name[0],
    modalities=ImageMaskModalities.CT_RTSTRUCT
)
nsclsc_rtstruct

ImageMaskInput<
	num_cases=2,
	dataset_name='NSCLC-Radiomics',
	modalities=<CT,RTSTRUCT>,
	output_streams=['CT', 'RTSTRUCT_CT'],
	series_col_names=['series_CT', 'series_RTSTRUCT_CT'],
>

In [11]:
nsclsc_seg = ImageMaskInput(
    dir_path=testdata / datasets_name[0],
    modalities=ImageMaskModalities.CT_SEG
)
nsclsc_seg

ImageMaskInput<
	num_cases=2,
	dataset_name='NSCLC-Radiomics',
	modalities=<CT,SEG>,
	output_streams=['CT', 'SEG'],
	series_col_names=['series_CT', 'series_SEG'],
>

## Using the Input Datasets

In [12]:
# List the case IDs (subject IDs)
nsclsc_rtstruct.keys()

['0_LUNG1-002', '1_LUNG1-001']

### Load a case

### StructureSets are automatically converted to `imgtools.Segmentation` objects

In [15]:
# by case ID or index
case = nsclsc_rtstruct['0_LUNG1-002']
#view
case

ImageMask(scan=Scan<
	BodyPartExamined=LUNG
	SliceThickness=3.00000
	PatientPosition=HFS
	Manufacturer=CMS, Inc.
	RescaleSlope=1
	ManufacturerModelName=XiO
	PixelSize=('0.9770', '0.9770', '3.00000')
	KVP=None, size=(512, 512, 111)
	spacing=(0.977, 0.977, 3.0)
	origin=(-250.112, -250.112, -133.4)
	direction=(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
>, mask=<Segmentation with ROIs: {'Esophagus': 1, 'GTV-1': 2, 'Heart': 3, 'Lung-Left': 4, 'Lung-Right': 5, 'Spinal-Cord': 6}>)

In [16]:
image, mask = case
mask

<Segmentation with ROIs: {'Esophagus': 1, 'GTV-1': 2, 'Heart': 3, 'Lung-Left': 4, 'Lung-Right': 5, 'Spinal-Cord': 6}>

### Whereas native `SEG` modalities get loaded as `Segmentation` objects

In [ ]:
# by case ID or index
caseid = nsclsc_seg.keys()[0]
image, mask  = nsclsc_seg[caseid]
mask

<Segmentation with ROIs: {'label_1': 1}>

### MR & RTSTRUCT example

In [19]:
case_id = vs_seg.keys()[0]
image, mask = vs_seg[case_id]
mask

<Segmentation with ROIs: {'*Skull': 1, 'TV': 2, 'cochlea': 3}>